<a href="https://colab.research.google.com/github/StratosFair/Mean_Escape_Time/blob/main/Duffin_oscillator/FEM/Duffin_Oscillator_fenics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-release-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

'try:\n    import dolfin\nexcept ImportError:\n    !wget "https://fem-on-colab.github.io/releases/fenics-install-release-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"\n    import dolfin'

In [ ]:
# try:
#     import gmsh
# except ImportError:
#     !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
#     import gmsh

In [ ]:
# !pip install meshio
# !apt-get install gmsh
# !gmsh --version
# !pip install --upgrade gmsh

In [ ]:
import fenics as fe
import mshr as msh
from matplotlib import pyplot

#  Duffing oscillator process in a ball of $\mathbb{R}^2$ : computation of solutions using Finite Element Method

Based on the paper [A neural network solution of first-passage problems](https://link.springer.com/article/10.1007/s10483-024-3189-8) (Jiamin Qian, Lincong Chen & J. Q. Sun, Oct. 2024), the 2-dimensional Duffing oscillator is defined by:
$$ d \begin{pmatrix} X_1(t)\\
X_2(t) \end{pmatrix} = \begin{pmatrix} X_2\\
-X_1 - X_1^3 - 2\zeta X_2 \end{pmatrix} dt + \begin{pmatrix} \sqrt{2\varepsilon}dB_1(t)\\
\sqrt{2\zeta} dB_2(t) \end{pmatrix}$$

The infinitesimal generator of this process is given for sufficiently smooth $f$ by
$$\mathscr Lf: x \mapsto b(x) \cdot \nabla f(x) + a(x) : \nabla^2 f(x)$$
where
$$b : x = (x_1, x_2)^T \mapsto \begin{pmatrix} x_2\\ -x_1 - x_1^3 - 2\zeta x_2 \end{pmatrix}  $$
and
$$a:x \mapsto \frac12 \sigma(x)\sigma(x)^T = \begin{pmatrix} \varepsilon & 0\\ 0 & \zeta\end{pmatrix} $$

## 1) The PDE problem

The Mean Escape Time (MET) $\tau$ is solution of the following elliptic problem

$$\begin{cases}\mathscr L\tau = -1 \quad \text{in } \Omega,\\
\tau= 0 \quad\text{ on }\partial\Omega\end{cases} $$

##2) The variational formulation

Because $\mathscr L$ is a second-order elliptic operator, we need to use integration by parts to make the PDE solvable in a traditional FEM solver. To do so, we multiply the PDE with a test function $v\in \hat V$, which belongs in the test function space $\hat V$. We then integrate over $\Omega$:
$$\begin{align*}\int_\Omega\mathscr L\tau\cdot v = -\int_\Omega v &\implies \int_\Omega \Big(x_2\partial_{x_1}\tau(x) - (x_1 + x_1^3 + 2\zeta x_2)\partial_{x_2} \tau(x) + \varepsilon \partial^2_{x_1,x_1}\tau(x) + \zeta \partial^2_{x_2,x_2}\tau(x) + 1\Big)v(x)dx = 0 \end{align*} $$

The first order derivatives of $\tau$ can be handled just fine by the solver, but the second order derivatives can't, hence we apply integration by parts to get the following weak formulation:

$$\int_\Omega \Big[\Big(x_2\partial_{x_1}\tau(x) - (x_1 + x_1^3 + 2\zeta x_2)\partial_{x_2} \tau(x)\Big)v(x) - \varepsilon \partial_{x_1}\tau(x)\partial_{x_1}v(x) - \zeta \partial_{x_2}\tau(x)\partial_{x_2}v(x)\Big] dx = - \int_\Omega v(x) dx$$

Which can be equivalently rewritten as

$$ a[\tau, v] = L(v) $$

Where $a$ is the bilinear form defined as

$$a : (\tau, v)\mapsto \int_\Omega \Big[\Big(x_2\partial_{x_1}\tau(x) - (x_1 + x_1^3 + 2\zeta x_2)\partial_{x_2} \tau(x)\Big)v(x) - \varepsilon \partial_{x_1}\tau(x)\partial_{x_1}v(x) - \zeta \partial_{x_2}\tau(x)\partial_{x_2}v(x)\Big] dx, $$

which equivalently reads

$$a : (\tau, v)\mapsto \int_\Omega \nabla\tau(x) \cdot b(x) v(x) - \text{diag}(\varepsilon, \zeta)\nabla\tau(x)\cdot\nabla v(x) dx$$

and $L$ is the linear form defined as

$$L : v\mapsto - \int_\Omega v(x) dx $$

## 3) FEniCS implementation

In [ ]:
#generate mesh
radius = 2
domain = msh.Circle(fe.Point(0, 0), radius)
mesh = msh.generate_mesh(domain, 64)
V = fe.FunctionSpace(mesh, "P", 1)

